Scraping for twitter

In [11]:
from twitter_scraper_selenium import scrape_keyword
#scrap 10 posts by searching keyword "india" from date 30th August till date 31st August
cancer = scrape_keyword(keyword="#cancer #nonsmoking", browser="chrome",
                      tweets_count=1,output_format="json" ,until="2023-08-31", since="2021-08-30")
print(cancer)

[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST driver version for 107.0.5304
[WDM] - Driver [/Users/sunayanashivanagi/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


 
{"1496367405852991490": {"tweet_id": "1496367405852991490", "username": "Emedkit_Medicin", "name": "Emedkit", "profile_picture": "https://pbs.twimg.com/profile_images/1358018215746625536/ldZsnUhV_x96.jpg", "replies": 1, "retweets": 0, "likes": 4, "is_retweet": false, "posted_time": "2022-02-23T06:12:45+00:00", "content": "Did you know fully half of women diagnosed with lung cancer have never smoked? \nDoctors are aware of this startling fact and continue to study why this trend happens to women who are never smokers.\n\n#lungcancer #lungs #lung #women #neversmoker #nonsmoking #Cancer #emedkit", "hashtags": ["lungcancer", "lungs", "lung", "women", "neversmoker", "nonsmoking", "Cancer", "emedkit"], "mentions": [], "images": ["https://pbs.twimg.com/media/FMQqOYRWQAU0Dgl?format=jpg&name=medium"], "videos": [], "tweet_url": "https://twitter.com/Emedkit_Medicin/status/1496367405852991490", "link": ""}}


Database Connection

In [10]:
import sqlite3

conn = sqlite3.connect('cancer.db')

try:
    conn.cursor()
    print("DB Connected");
except Exception as ex:
    print("DB not Connected");

DB Connected


Tweet Url


In [ ]:
conn = sqlite3.connect('test.db')
print ("Opened database successfully");
conn.execute('''drop table COMPANY;''')
conn.execute('''CREATE TABLE if not exists COMPANY
         (ID INTEGER PRIMARY KEY AUTOINCREMENT,
         NAME           TEXT    NOT NULL,
         AGE            INT     NOT NULL,
         ADDRESS        CHAR(50),
         SALARY         REAL);''')

conn.execute('''''')
print ("Table created successfully");

conn.close()